In [1]:
import sys
import os
import platform
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy


flopy is installed in f:\Anaconda3\lib\site-packages\flopy


In [2]:
curpath = os.getcwd()
curpath

'F:\\FloPy_demo'

In [3]:
#Set the paths
curpath = os.getcwd()
modelpth=os.path.join(curpath,'mf2005')
filehds=os.path.join(modelpth, 'demo_pred.hds')
cbdf=os.path.join(modelpth, 'demo_pred.cbd')
fcsv=os.path.join(modelpth, 'Demo_results_at_the targets.csv')
Realization='Demo 2'

### Setup basic info for flopy

In [4]:

#Set name of MODFLOW exe
#  assumes executable is in users path statement
version = 'mf2005'
exe_name = 'mf2005'
if platform.system() == 'Windows':
    exe_name = 'mf2005.exe'
mfexe = exe_name




#make sure modelpth directory exists
if not os.path.exists(modelpth):
    os.makedirs(modelpth)
    
print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
numpy version: 1.16.5
matplotlib version: 3.1.1
flopy version: 3.3.0


### load the MODFLOW model


There are some issues to load the EAA bottom-up run with plopy. So I decided to use the USGS scenario simulation run by replacing the RCH file from the EAA bottom-up run because I just want to check the recharge rate applied to the four cells.

In [ ]:
ml = flopy.modflow.Modflow.load('demo_pred.nam', model_ws=modelpth, 
                                exe_name=exe_name, version=version)
#ml.model_ws = modelpth


### Show basic info for the model

In [ ]:
ml

### List the packages

In [ ]:
ml.get_name_file_entries().split('\n')

### load the cells at the four targets (J17, J27, San Marcos Springs, and Comal Springs)


The irow and jcolum for the two index wells are givien in the file by USGS
  D:\USGS_UA\working\_data\head_obs.crd.csv
  
  The name (or state well number) in the csv file 
    6837203 for J17
    6950302 for J27
  note that row and col are started from 0

In [ ]:
### this file is provided with USGS Uncertainty analysis for the EAA MODFLOW model. 
### USGS used this file to include all the water head observation data and
### also provide the col and row for each observation well
### the row and col are numbered from 0.
targheadcell=pd.read_csv('head_obs.crd.csv')

In [ ]:
targheadcell.head()

In [ ]:
targheadcell.columns=['indx','name','x','y','layer','row','col']

In [ ]:
targheadcell[targheadcell['name']=='6837203']    # this for J17

In [ ]:
targheadcell[targheadcell['name']=='6950302']    # this for J27

Load the cells for the drain to represent springs 

In [ ]:
### this file is provided with USGS Uncertainty analysis for the EAA MODFLOW model. 
### USGS used this file to include cells for each of the EAA springs 
### the row and col are numbered from 0.
targsprgcell=pd.read_csv('DRN_dict.csv',header=None,delimiter=r"\s+")

In [ ]:
targsprgcell.columns=['name','layer','row','col']     

In [ ]:
targsprgcell

### load simulated water heads from the hds file for the J17 and J27

In [ ]:
def extracttargethead(ml,filehds,targheadcell):
        #filehds = os.path.join(modelpth,'eaa_pred.hds')
        hds = flopy.utils.HeadFile(filehds)
        row_j17=targheadcell[targheadcell['name']=='6837203']['row'].values[0]
        col_j17=targheadcell[targheadcell['name']=='6837203']['col'].values[0]

        row_j27=targheadcell[targheadcell['name']=='6950302']['row'].values[0]
        col_j27=targheadcell[targheadcell['name']=='6950302']['col'].values[0]

        print(row_j17,col_j17,row_j27,col_j27)

        head_j17=[]
        head_j27=[]
        for kper in range (ml.nper):
            kstpker=[0,kper]
            h=hds.get_data(kstpkper=kstpker)
            head_j17.append(h[0][row_j17][col_j17])
            head_j27.append(h[0][row_j27][col_j27])   

        return head_j17,head_j27 
             

In [ ]:


SR_j17,SR_j27= extracttargethead(ml,filehds,targheadcell)



### load the drain file and read simulated spring dsicharges at the target cells

Note that the unit for the drain ouput (*.cbd) is ft3/day with a negtaibe sign. Unit for the observed discharge rate is ft3/s with a positive sign. The values read from the output file need to be converted by divided the factor of -24*60*60.

In [ ]:
### the following is to get the row and col for the San Marcos Springs and Comal springs. 
### NOte thatthe Modflow model starts the row and the col from 1, but FloPy from 0.
def extracttargetspring(ml,cbdf,targsprgcell):
    
    row_sm=targsprgcell[targsprgcell['name']=='SanMar']['row'].values[0]
    col_sm=targsprgcell[targsprgcell['name']=='SanMar']['col'].values[0]

    row_comal=targsprgcell[targsprgcell['name']=='Comal']['row'].values[0]
    col_comal=targsprgcell[targsprgcell['name']=='Comal']['col'].values[0]

    print(row_sm,col_sm,row_comal,col_comal)

    cbd = flopy.utils.CellBudgetFile(cbdf,precision='double')

    flx_sm=[]
    flx_comal=[]
    for kper in range (ml.nper):
        kstpker=[0,kper]
        flx=cbd.get_data(kstpkper=kstpker)
        flx_sm.append(flx[0][0][row_sm][col_sm]/(-24*60*60))
        flx_comal.append(flx[0][0][row_comal][col_comal]/(-24*60*60))  
    
    return flx_sm,flx_comal

In [ ]:


SR_sm,SR_comal= extracttargetspring(ml,cbdf,targsprgcell)


### load the observed targets from the archived bottom up runs

two files are stored in the following folder
D:\Bottom-Up Archive\Bottom-Up Model Runs\L4STG5_MarcTrigger_newASRSchedule_AddVISPO_for30cfs

target_flux.xlsx    # this is for the spring discharges
target_head.xlsx    # this is for the observed groundwater head 




first let us load the water head

In [ ]:
xls_name='target_head.xlsx'
J17_sheet_name='J-17'
J27_sheet_name='J-27'

In [ ]:
ori_J17 = pd.read_excel(xls_name, sheet_name = J17_sheet_name)
ori_J27 = pd.read_excel(xls_name, sheet_name = J27_sheet_name)
ob_j17=ori_J17['Observed'].to_list()
ob_j27=ori_J27['Observed'].to_list()



let us load the spring dischare rates for the springs

In [ ]:
xls_name='target_flux.xlsx'
comal_sheet_name='Comal'
sm_sheet_name='SanMarcos'

In [ ]:
ori_comal = pd.read_excel(xls_name, sheet_name = comal_sheet_name)
ori_sm = pd.read_excel(xls_name, sheet_name = sm_sheet_name)
ob_comal=ori_comal['Observed'].to_list()
ob_sm=ori_sm['Observed'].to_list()


In [ ]:
date=pd.date_range(start='1/31/1947', end='12/31/1958', freq='M') #.to_period('M')

In [ ]:
print(len(date))
print(len(ob_j17[0:144]),len(ob_j27),len(ob_comal),len(ob_sm))
print(len(SR_j17),len(SR_j27),len(SR_comal),len(SR_sm))
#print(len(r_j17_lst),len(r_j27_lst),len(r_comal_lst),len(r_sanmar_lst))


In [ ]:
### the first column is date, the following four columns are observed and the last four columns are simulated
df=pd.DataFrame({'date':date,'ob_j17_ft':ob_j17,'ob_j27_ft':ob_j27,'ob_comal_cfs':ob_comal,'ob_sm_cfs':ob_sm,
                'simh_j17_ft':SR_j17,'simh_j27_ft':SR_j27,'simq_comal_cfs':SR_comal,'simq_sm_cfs':SR_sm})

In [ ]:
df.set_index("date", inplace = True)   # reset the index with datetimeindex

In [ ]:
df.head()

In [ ]:
df.to_csv(fcsv)

In [ ]:
df.tail()

### Plot the results 

In [ ]:
def drawplotts(df,obcol,simcol,title,ylabel,xlabel,ylim):
    fig,ax=plt.subplots(figsize=(14,6))
    df[simcol].plot(color='b',linestyle='-',marker='',label='Simulated',legend=True,ax=ax)
    df[obcol].plot(color='r',linestyle='',marker='o',label='Observed',legend=True,ax=ax)
    ax.tick_params(axis='both', which='major', labelsize=14)
    plt.title(title,fontsize=14)
    ax.set_ylabel(ylabel,fontsize=14)
    ax.set_xlabel(xlabel,fontsize=14)    
    ax.set_ylim(ylim)   

In [ ]:
### for water head at J17
obcol='ob_j17_ft'
simcol='simh_j17_ft'
title='Comparison of water heads at the J17 for the USGS '+ Realization
ylabel='Groundwater head (ft)'
xlabel='Date'
ylim=[580,700]
drawplotts(df,obcol,simcol,title,ylabel,xlabel,ylim)

In [ ]:
### for water head at J27

obcol='ob_j27_ft'
simcol='simh_j27_ft'
title='Comparison of water heads at the J27 for the USGS '+ Realization
ylabel='Groundwater head (ft)'
xlabel='Date'
ylim=[700,900]
drawplotts(df,obcol,simcol,title,ylabel,xlabel,ylim)

In [ ]:
### for spring flows at Comal springs
obcol='ob_comal_cfs'
simcol='simq_comal_cfs'
title='Comparison of spring dsicahrges  at the Comal Springs for the USGS '+ Realization
ylabel='Spring discharge (cfs)'
xlabel='Date'
ylim=[0,500]
drawplotts(df,obcol,simcol,title,ylabel,xlabel,ylim)

In [ ]:
### for spring flows at San Marcos springs
obcol='ob_sm_cfs'
simcol='simq_sm_cfs'
title='Comparison of spring dsicahrges  at the San Marocos Springs for the USGS '+ Realization
ylabel='Spring discharge (cfs)'
xlabel='Date'
ylim=[40,350]
drawplotts(df,obcol,simcol,title,ylabel,xlabel,ylim)

In [ ]:
#### plot the 1to1comparison

In [ ]:
def drawsubplots(ax,x,y,xline,title='a)',xlabel='observed',ylabel='simulated',lgd=False):
    
    ax.plot(xline,xline,color='r',linestyle='-',label='1:1 line')
    ax.plot(x,y, color='b',linestyle='',marker='o',label='Simulated')
    ax.set_title(title,fontsize=16)
    ax.set_ylabel(ylabel,fontsize=16)
    ax.set_xlabel(xlabel,fontsize=16)
    if lgd==True:
        ax.legend(loc='best',ncol=1,fontsize=16)
    ax.tick_params(axis="x", labelsize=16)
    ax.tick_params(axis="y", labelsize=16)

In [ ]:
fig,axes=plt.subplots(2,2,figsize=(14,14))
## plot J17
ax=axes[0,0]
x=df['ob_j17_ft'].values
y=df['simh_j17_ft'].values
xline=[600,700]
title='a) J17'
xlabel='observed head(ft)'
ylabel='simulated head(ft)'
lgd=False
drawsubplots(ax,x,y,xline,title,xlabel,ylabel,lgd)   

    ## plot J27
ax=axes[0,1]
x=df['ob_j27_ft'].values
y=df['simh_j27_ft'].values
xline=[800,900]
title='b) J27'
xlabel='observed head(ft)'
ylabel='simulated head(ft)'
lgd=False
drawsubplots(ax,x,y,xline,title,xlabel,ylabel,lgd)   

    
## plot Comal springs
ax=axes[1,0]
x=df['ob_comal_cfs'].values
y=df['simq_comal_cfs'].values
xline=[0,500]
title='c) Comal Springs'
xlabel='observed discharge(cfs)'
ylabel='simulated discharge(cfs)'
lgd=False
drawsubplots(ax,x,y,xline,title,xlabel,ylabel,lgd)   
   

## plot San Marcos springs
ax=axes[1,1]
x=df['ob_sm_cfs'].values
y=df['simq_sm_cfs'].values
xline=[0,500]
title='d) San Marcos Springs'
xlabel='observed discharge(cfs)'
ylabel='simulated discharge(cfs)'
lgd=True
drawsubplots(ax,x,y,xline,title,xlabel,ylabel,lgd)   


